# 학습 목표
1. 구축된 DTI 모델을 이용해 ZINC 데이터베이스를 가상 스크리닝 한다
2. 가상 스크리닝 후 Docking을 통해 추가적으로 검증한다.

# Load Pretrained Model

In [ ]:
!pip install transformers
!gdown --id 1VYo2WaUz3ZBlhqt7QvmklkalqxeBAMWq # model
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina # gnina
!chmod +x gnina

from transformers import TFBertModel, AutoTokenizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, concatenate, Conv2D, Flatten, Dense, Dropout, BatchNormalization, Activation, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.backend import expand_dims
from transformers import TFBertModel, AutoTokenizer
import numpy as np

protein_BERT_path = "Rostlab/prot_bert"
protein_encoder = TFBertModel.from_pretrained(protein_BERT_path, from_pt=True)
protein_tokenizer = AutoTokenizer.from_pretrained(protein_BERT_path)

smiles_BERT_path = "seyonec/PubChem10M_SMILES_BPE_396_250"
smiles_tokenizer = AutoTokenizer.from_pretrained(smiles_BERT_path)
smiles_bert_encoder = TFBertModel.from_pretrained(smiles_BERT_path, from_pt=True)

max_len = 512
LAMBDA = 0.1

def mixBERT():
    # chemical
    smiles_input_ids = Input(shape=(max_len, ), dtype=tf.int32, name="smiles_input_ids")
    smiles_embedding = smiles_bert_encoder(smiles_input_ids)[0]
    smiles_clf_output = smiles_embedding[:, 0, :]
    avg = tf.keras.layers.AveragePooling2D(pool_size=(512-1, 1), strides=(1, 1))(expand_dims(smiles_embedding[:, 1:, :]))
    avg = tf.reshape(avg, [-1, 768])

    fp = Input(shape=(2048, ), dtype=tf.float32, name="fp")

    # protein
    protein_input_ids = Input(shape=(max_len, ), dtype=tf.int32, name="protein_input_ids")
    protein_embedding = protein_encoder(protein_input_ids)[0]
    protein_clf_output = protein_embedding[:, 0, :]
    avg2 = tf.keras.layers.AveragePooling2D(pool_size=(512-1, 1), strides=(1, 1))(expand_dims(protein_embedding[:, 1:, :]))
    avg2 = tf.reshape(avg2, [-1, 1024])

    # concatenate
    net = concatenate([smiles_clf_output, avg, fp, protein_clf_output, avg2], axis=1)

    # dense
    x = layers.Dense(5632, kernel_regularizer=keras.regularizers.l2(LAMBDA))(net)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(2048, kernel_regularizer=keras.regularizers.l2(LAMBDA))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(1024, kernel_regularizer=keras.regularizers.l2(LAMBDA))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(512, kernel_regularizer=keras.regularizers.l2(LAMBDA))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(256, kernel_regularizer=keras.regularizers.l2(LAMBDA))(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.3)(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(1, activation='relu', kernel_regularizer=keras.regularizers.l2(LAMBDA))(x)

    model = Model([smiles_input_ids, fp, protein_input_ids], x)
    model.layers[2].trainable = False
    model.layers[3].trainable = False

    return model
model = mixBERT()
model.load_weights('./model.h5')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1VYo2WaUz3ZBlhqt7QvmklkalqxeBAMWq
To: /content/model.h5
100% 2.57G/2.57G [00:19<00:00, 129MB/s]
--2023-05-30 11:31:02--  https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubuserc

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['roberta.encoder.layer.2.attention.output.dense.weight', 'roberta.encoder.layer.1.attention.self.query.bias', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.5.output.LayerNorm.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.5.attention.self.value.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.dense.bias', 'roberta.encoder.layer.4.output.dense.bias', 'roberta.embeddings.word_embeddings.weight', 'roberta.encoder.layer.4.output.LayerNorm.weight', 'roberta.encoder.layer.5.attention.output.LayerNorm.bias', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.outpu

In [ ]:
!pip install rdkit

from transformers import TFBertModel, AutoTokenizer
from rdkit.Chem import AllChem, Lipinski, Descriptors, Crippen
from rdkit import Chem, DataStructs
import numpy as np

def bertEncoder():
    protein_input_ids = layers.Input(shape=(max_len, ), dtype=tf.int32, name="protein_input_ids")
    protein_embedding = bert_encoder(protein_input_ids)[0]
    protein_clf_output = protein_embedding[:, 0, :]
    return keras.Model(protein_input_ids, protein_clf_output)

def protein_encoding(protein):
    protein = list(map(lambda x: ' '.join(list(x)), protein))
    protein = np.array(protein_tokenizer(list(protein), padding='max_length', max_length=512, truncation=True)['input_ids'])
    return np.array(protein)

def smiles_encoding(smiles):
    smiles = np.array(smiles_tokenizer(list(smiles), padding='max_length', max_length=512, truncation=True)['input_ids'])
    return smiles

def smiles2fp(smiles):
    res = list()
    for smile in smiles:
        mol = Chem.MolFromSmiles(smile)
        fp_obj = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048,
                                                   useChirality=True)
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp_obj, arr)
        res.append(arr)
    return np.array(res)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 47.7 MB/s eta 0:00:00


# 환경변수 설정

In [ ]:
max_len = 512
BATCH_SIZE = 64
EPOCHS = 2
select_number = 1000

protein_sequence = 'MAKTVAYFYDPDVGNFHYGAGHPMKPHRLALTHSLVLHYGLYKKMIVFKPYQASQHDMCRFHSEDYIDFLQRVSPTNMQGFTKSLNAFNVGDDCPVFPGLFEFCSRYTGASLQGATQLNNKICDIAINWAGGLHHAKKFEASGFCYVNDI\
VIGILELLKYHPRVLYIDIDIHHGDGVQEAFYLTDRVMTVSFHKYGNYFFPGTGDMYEVGAESGRYYCLNVPLRDGIDDQSYKHLFQPVINQVVDFYQPTCIVLQCGADSLGCDRLGCFNLSIRGHGECVEYVKSFNIPLLVLGGGGYTVRNVARCWTYETSLLVEEAISE\
ELPYSEYFEYFAPDFTLHPDVSTRIENQNSRQYLDQIRQTIFENLKMLNHAPSVQIHDVPADLLTYDRTDEADAEERGPEENYSRPEAPNEFYDGDHDNDKESDVEI'
protein_name = '4A69'
ligand_name = 'I0P'

!wget http://files.rcsb.org/download/{protein_name}.pdb
!grep ATOM {protein_name}.pdb > rec.pdb
!grep {ligand_name} {protein_name}.pdb > lig.pdb

--2022-01-12 00:54:54--  http://files.rcsb.org/download/4A69.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.110
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘4A69.pdb’

4A69.pdb                [   <=>              ] 670.70K   898KB/s    in 0.7s    

2022-01-12 00:54:55 (898 KB/s) - ‘4A69.pdb’ saved [686799]



# Virtual Screening

## Get chemical library

In [ ]:
# ZINC clean dataset (250k)
!wget https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv

--2022-01-12 00:54:55--  https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22606589 (22M) [text/plain]
Saving to: ‘250k_rndm_zinc_drugs_clean_3.csv’

250k_rndm_zinc_drug 100%[===================>]  21.56M  --.-KB/s    in 0.06s   

2022-01-12 00:54:58 (333 MB/s) - ‘250k_rndm_zinc_drugs_clean_3.csv’ saved [22606589/22606589]



In [ ]:
import pandas as pd
zinc_df = pd.read_csv('./250k_rndm_zinc_drugs_clean_3.csv')
drug = zinc_df['smiles']
drug = list(map(lambda x: x.strip(), drug))
protein = [protein_sequence] * len(drug)
zinc_df

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
...,...,...,...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc...,3.36790,0.745901,2.900726
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.87430,0.799426,2.326627
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1\n,2.90054,0.878086,1.840642
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1\n,2.55624,0.852917,2.024638


In [ ]:
import math
import time
import datetime

start = time.time()
zinc_df['result'] = model.predict((smiles_encoding(drug), smiles2fp(drug), protein_encoding(protein)), batch_size=16)
zinc_df = zinc_df.sort_values('result', ascending=True).reset_index(drop=True) # select_number
zinc_df = zinc_df[:select_number]
zinc_df['score'] = ''
end = time.time()

sec = (end - start)


result_list = str(datetime.timedelta(seconds=sec)).split(".")
print("Processing time: ", result_list[0])

Processing time:  3:40:14


## SMILES-2-SDF and Docking(GNINA)

In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os
from tqdm.notebook import tqdm

for i in tqdm(range(select_number)):


    m = Chem.MolFromSmiles(zinc_df['smiles'][i])
    with Chem.SDWriter(f'try.sdf') as w:
        w.write(m)
    os.system('./gnina -r rec.pdb -l try.sdf --autobox_ligand lig.pdb -o docked.sdf --seed 0 > res.txt')
    os.system("grep -n '    1' res.txt > res2.txt")

    f = open("./res2.txt", 'r')
    line = f.readline()
    f.close()
    affinity = line.split()[2]
    zinc_df['score'][i] = affinity

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
zinc_df

,smiles,logP,qed,SAS,result,score
0,COc1ccc(S(=O)(=O)N2CCN(C(=O)c3cc4cc(Cl)ccc4[nH...,2.97660,0.678520,2.115870,0.558186,-2.95
1,COc1cc2[nH]c(C(=O)N3CCN(c4ccc(F)cc4)CC3)cc2c(O...,3.29520,0.683365,2.277065,0.559837,-5.16
2,COc1cccc(N2CCN(C(=O)c3cc4cc(Br)ccc4[nH]3)CC2)c1\n,3.90140,0.684185,2.096046,0.560809,-6.30
3,COc1ccc(OC)c(CN2CCN(C(=O)c3cc4ccc(OC)cc4[nH]3)...,3.15170,0.668185,2.153996,0.564226,-6.08
4,CC(=O)c1ccc(N2CCN(C(=O)c3cc4cc(Cl)ccc4[nH]3)CC...,3.98630,0.673428,2.076513,0.566044,-3.66
...,...,...,...,...,...,...
995,C[C@@H](C(=O)[O-])N1CC[C@@]2(CCC1=O)Nc1ccccc1S...,-0.76240,0.754759,4.536712,0.657600,-5.73
996,COc1cccc2[nH]c(C(=O)N3CCC[C@@H](c4nnc(-c5ccsc5...,4.30780,0.521486,3.035645,0.657660,-5.03
997,C[C@@H]1CS(=O)(=O)N(c2cccc(C(=O)N3CCc4ccccc4C3...,2.19760,0.797890,2.814664,0.657677,-7.98
998,COc1cc(Cl)c([N-]S(=O)(=O)c2c(C(=O)N3CCCC3)c(C)...,3.94264,0.636299,3.212450,0.657680,-6.45
